# Create Main Data

This notebook creates the main data table in spark. 

In [0]:
%pip install -U google-genai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# imports & config

import json
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.ml.pipeline import PipelineModel
import matplotlib.pyplot as plt


PAIR_TABLE = "default.listing_upcoming_events"

ARTIFACT_BASE = "/Workspace/Users/odelia.dov@campus.technion.ac.il/20260125_170454" 

CONFIG_JSON_PATH = f"{ARTIFACT_BASE}/config.json"

# widgets 
dbutils.widgets.removeAll()

dbutils.widgets.text("listing_id", "", "Listing ID")
dbutils.widgets.text("gemini_api_key", "", "Gemini API key (will be visible)")  # Databricks doesn't have password widgets
dbutils.widgets.dropdown(
    "gemini_model",
    "gemini-2.5-flash",
    ["gemini-2.5-flash"],
    "Gemini model"
)

In [0]:
# load the pricing model

def assert_dbfs_exists(path: str, what: str = ""):
    # function to asses if the path exists to load the pricing mode.
    try:
        _ = dbutils.fs.ls(path)
        print(f" Found {what or path}: {path}")
    except Exception as e:
        raise FileNotFoundError(f"Missing {what or 'path'}: {path}\nOriginal error: {e}")


# Load config.json
cfg = json.loads(dbutils.fs.head(CONFIG_JSON_PATH, 10_000_000))

PRICE_MODEL_PATH = cfg["artifacts"]["price_model_path"]

# optional check:
assert_dbfs_exists(PRICE_MODEL_PATH, "price_model_path")

# actual load of the price model:
price_model = PipelineModel.load(PRICE_MODEL_PATH)


 Found price_model_path: /Workspace/Users/odelia.dov@campus.technion.ac.il/20260125_170454/price_model


In [0]:
# Load the pair df (listing-event pairs)

pair_df = spark.table(PAIR_TABLE)


In [0]:
# important cols

COL_LISTING_ID     = "listing_id"
COL_CURRENT_PRICE  = "total_price"
COL_DISTANCE_KM    = "distance_km"
COL_EVENT_TYPE     = "event_type"
COL_DAYS_UNTIL     = "days_until_event"
COL_VENUE_CAP      = "venue_capacity"

LOOKAHEAD_DAYS = cfg.get("params", {}).get("lookahead_days", 90)

In [0]:
# Code to get a df with the prices

def score_pairs(df: DataFrame) -> DataFrame:

    ml_df = (
        df
        .withColumn("current_price", F.round(F.col(COL_CURRENT_PRICE).cast("double"), 2))
        .withColumn(COL_DISTANCE_KM, F.col(COL_DISTANCE_KM).cast("double"))
        .withColumn(
            COL_DAYS_UNTIL,
            F.coalesce(F.col(COL_DAYS_UNTIL), F.lit(LOOKAHEAD_DAYS)).cast("double")
        )
        .withColumn(
            COL_VENUE_CAP,
            F.coalesce(F.col(COL_VENUE_CAP), F.lit(0.0)).cast("double")
        )
        .dropna(subset=[COL_EVENT_TYPE, COL_DISTANCE_KM, "current_price"])
    )

    
    scored = price_model.transform(ml_df).withColumn(
        "suggested_price", F.round(F.col("prediction"), 2)
    )

    # rules for price to not be lower than current
    scored = scored.withColumn(
        "suggested_price",
        F.when(F.col("suggested_price") < 0, F.lit(0.0))
         .when(F.col("suggested_price") < F.col("current_price"), F.col("current_price"))
         .otherwise(F.col("suggested_price"))
    )

    # get the delta
    scored = scored.withColumn(
        "price_delta",
        F.col("suggested_price") - F.col("current_price")
    )

    return scored
scored_df = score_pairs(pair_df)

In [0]:
scored_df.count()

14259910

In [0]:
spark.sql("""
SELECT DISTINCT listing_id
FROM default.listing_upcoming_events
LIMIT 10
""").show()


+-------------------+
|         listing_id|
+-------------------+
|1120110109263071637|
|1267106355022254426|
|1284238239006581682|
|           28205849|
|           53770597|
|           40326158|
|1263268204266298643|
|1149470810034756340|
|           49347683|
|           33531288|
+-------------------+



In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# If you have a spark table instead of a variable, uncomment:
# scored_df = spark.table("YOUR_SCORED_TABLE_NAME")

listing_id = dbutils.widgets.get("listing_id").strip()

if not listing_id:
    print("Enter a listing_id in the widget above, then rerun this cell.")
else:
    # filter + add row index
    w = Window.orderBy(
        F.col("event_date").asc_nulls_last(),
        F.col("event_name").asc_nulls_last(),
        F.col("venue_name").asc_nulls_last()
    )

    events_df = (
        scored_df
        .filter(F.col("listing_id") == listing_id)
        .withColumn("row_idx", F.row_number().over(w) - 1)
        .select(
            "row_idx",
            "event_name",
            "event_type",
            "event_date",
            "venue_name",
            F.round(F.col("distance_km"), 2).alias("distance_km"),
            F.col("days_until_event").alias("days_until"),
            F.round(F.col("current_price"), 2).alias("current_price"),
            F.round(F.col("suggested_price"), 2).alias("suggested_price_for_event"),
            "description"  # keep for generation, we won't display it by default
        )
    )

    cnt = events_df.count()
    if cnt == 0:
        print("No events found for that listing_id.")
    else:
        print(f"Found {cnt} rows for listing_id={listing_id}")
        display(
            events_df.select(
                "row_idx",
                "event_name",
                "event_type",
                "event_date",
                "venue_name",
                "distance_km",
                "days_until",
                "current_price",
                "suggested_price_for_event"
            )
        )

        # Build / refresh the row selector widget based on found rows
        idxs = [str(r["row_idx"]) for r in events_df.select("row_idx").distinct().orderBy("row_idx").collect()]
        # Remove & recreate so options update when listing_id changes
        try:
            dbutils.widgets.remove("event_row_idx")
        except:
            pass
        dbutils.widgets.dropdown("event_row_idx", idxs[0], idxs, "Choose event row (row_idx)")

        # Store events_df in a temp view so we can reuse it in the next cell without recomputing
        events_df.createOrReplaceTempView("tmp_events_for_listing")


Found 39 rows for listing_id=1120110109263071637


row_idx,event_name,event_type,event_date,venue_name,distance_km,days_until,current_price,suggested_price_for_event
0,Charlotte FC vs. Austin FC,Soccer,2026-03-07,Bank of America Stadium,3.78,41.0,144.0,1107.1
1,Charlotte FC vs. Austin FC,Soccer,2026-03-07,Bank of America Stadium,3.78,41.0,144.0,1107.1
2,Charlotte FC vs. Austin FC,Soccer,2026-03-07,Bank of America Stadium,3.79,41.0,144.0,1107.1
3,Charlotte FC vs. Inter Miami CF,Soccer,2026-03-14,Bank of America Stadium,3.78,48.0,144.0,1107.1
4,Charlotte FC vs. Inter Miami CF,Soccer,2026-03-14,Bank of America Stadium,3.78,48.0,144.0,1107.1
5,Charlotte FC vs. Inter Miami CF,Soccer,2026-03-14,Bank of America Stadium,3.79,48.0,144.0,1107.1
6,Charlotte FC vs. New York Red Bulls,Soccer,2026-03-21,Bank of America Stadium,3.78,55.0,144.0,1107.1
7,Charlotte FC vs. New York Red Bulls,Soccer,2026-03-21,Bank of America Stadium,3.78,55.0,144.0,1107.1
8,Charlotte FC vs. New York Red Bulls,Soccer,2026-03-21,Bank of America Stadium,3.79,55.0,144.0,1107.1
9,Charlotte FC vs. Philadelphia Union,Soccer,2026-04-04,Bank of America Stadium,3.78,69.0,144.0,1103.55


In [0]:
%pip install -U "pydantic>=2,<3"
dbutils.library.restartPython()


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pydantic
print(pydantic.__version__)


2.12.5


In [0]:
dbutils.widgets.text("gemini_api_key", "", "Gemini API key")
dbutils.widgets.text("gemini_model", "gemini-1.5-pro", "Gemini model")
dbutils.widgets.text("listing_id", "", "Listing ID")
dbutils.widgets.text("event_row_idx", "0", "Event row idx")


In [0]:
api_key = dbutils.widgets.get("gemini_api_key").strip()


In [0]:
spark.sql("SHOW TABLES").filter("tableName = 'tmp_events_for_listing'").show(truncate=False)


+--------+----------------------+-----------+
|database|tableName             |isTemporary|
+--------+----------------------+-----------+
|        |tmp_events_for_listing|true       |
+--------+----------------------+-----------+



In [0]:
dbutils.widgets.text("listing_id", "", "Listing ID")


In [0]:
listing_id = dbutils.widgets.get("listing_id").strip()
if not listing_id:
    raise ValueError("Set the listing_id widget first (top of notebook).")

tmp = spark.sql(f"""
SELECT
  *,
  row_number() over (order by event_date, venue_name, event_name) - 1 as row_idx
FROM default.listing_upcoming_events
WHERE listing_id = '{listing_id}'
""")

tmp.createOrReplaceTempView("tmp_events_for_listing")
display(tmp.select("row_idx", "event_name", "event_date", "venue_name", "distance_km"))


row_idx,event_name,event_date,venue_name,distance_km
0,2026 NFC Conference Championship,2026-01-25,Bank of America Stadium,3.784086484449908
1,2026 NFC Conference Championship,2026-01-25,Bank of America Stadium,3.776740786992762
2,2026 NFC Conference Championship,2026-01-25,Bank of America Stadium,3.7852406594448667
3,2025 Printing Fee,2026-01-31,Bank of America Stadium,3.784086484449908
4,2025 Printing Fee,2026-01-31,Bank of America Stadium,3.776740786992762
5,2025 Printing Fee,2026-01-31,Bank of America Stadium,3.7852406594448667
6,2025 Price Codes,2026-02-28,Bank of America Stadium,3.784086484449908
7,2025 Price Codes,2026-02-28,Bank of America Stadium,3.776740786992762
8,2025 Price Codes,2026-02-28,Bank of America Stadium,3.7852406594448667
9,Austin FC Hot Dog Package,2026-03-07,Bank of America Stad - GEN ADM,3.784086484449939


In [0]:
idxs = [str(r["row_idx"]) for r in spark.sql(
    "SELECT row_idx FROM tmp_events_for_listing ORDER BY row_idx"
).collect()]

if not idxs:
    raise ValueError("No events found for this listing_id.")

try:
    dbutils.widgets.remove("event_row_idx")
except:
    pass

dbutils.widgets.dropdown("event_row_idx", idxs[0], idxs, "Event row idx")


In [0]:
from google import genai

api_key = dbutils.widgets.get("gemini_api_key").strip()
model_name = dbutils.widgets.get("gemini_model").strip()
row_idx_str = dbutils.widgets.get("event_row_idx").strip()

if not api_key:
    raise ValueError("Paste your Gemini API key in the widget (gemini_api_key) and rerun.")

if not row_idx_str:
    raise ValueError("Choose an event row (event_row_idx) and rerun.")

row_idx = int(row_idx_str)

# Pull the selected row from the temp view created in the previous cell
sel_df = spark.sql(f"SELECT * FROM tmp_events_for_listing WHERE row_idx = {row_idx} LIMIT 1")
rows = sel_df.collect()
if not rows:
    raise ValueError("Selected row not found. Rerun the lookup cell first.")

r = rows[0].asDict()

# Build prompt (tight + anti-hallucination)
current_desc = (r.get("description") or "").strip()
if len(current_desc) > 1200:
    current_desc = current_desc[:1200].rstrip() + "..."

event_name = r.get("event_name", "")
event_type = r.get("event_type", "")
event_date = r.get("event_date", "")
venue_name = r.get("venue_name", "")
distance_km = r.get("distance_km", "")
days_until = r.get("days_until", "")
current_price = r.get("current_price", "")
suggested_price = r.get("suggested_price_for_event", "")

prompt = f"""
Rewrite this Airbnb listing description to appeal to guests attending the event below.

Rules:
- Do NOT add new facts (no invented amenities, rules, views, parking, neighborhood claims, etc.).
- 30–70 words, one paragraph.
- Natural, helpful tone (not salesy).
- Mention the event and convenience to the venue using the info below.

Event:
- Name: {event_name}
- Type: {event_type}
- Date: {event_date}
- Venue: {venue_name}
- Distance (km): {distance_km}
- Days until event: {days_until}

Pricing context (do not claim discounts or guarantees):
- Current price: {current_price}
- Suggested price: {suggested_price}

Original description:
{current_desc}

Improved description:
""".strip()

client = genai.Client(api_key=api_key)

resp = client.models.generate_content(
    model=model_name,
    contents=prompt
)

new_text = (resp.text or "").strip()

print("Generated description: \n")
print(new_text)

display({
    "listing_id": dbutils.widgets.get("listing_id"),
    "row_idx": row_idx,
    "event_name": event_name,
    "venue_name": venue_name,
    "generated_description": new_text
})


Generated description: 

Attending the 2026 NFC Conference Championship? This lovely 2-bedroom, 1-bath home in historic Plaza Midwood is perfectly located less than 4km from Bank of America Stadium. Enjoy easy stadium access, then unwind on the porch or by the fire pit. It's a comfortable and centrally-located base for your championship weekend.


{'listing_id': '1120110109263071637',
 'row_idx': 0,
 'event_name': '2026 NFC Conference Championship',
 'venue_name': 'Bank of America Stadium',
 'generated_description': "Attending the 2026 NFC Conference Championship? This lovely 2-bedroom, 1-bath home in historic Plaza Midwood is perfectly located less than 4km from Bank of America Stadium. Enjoy easy stadium access, then unwind on the porch or by the fire pit. It's a comfortable and centrally-located base for your championship weekend."}